## Adding Regularization with L2 and Dropout

In [1]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # ignore the GPU warning
tf.__version__

'2.6.0'

In [2]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [4]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [5]:
def my_model():
    inputs = tf.keras.Input(shape=(32,32,3))
    x = tf.keras.layers.Conv2D(32,3,padding= 'same',
                               kernel_regularizer = tf.keras.regularizers.l2(0.01))(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(64,5,padding='same',
                               kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Conv2D(128,3,
                               kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x) # regularization
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64,
                               kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(10)(x)
    
    model =tf.keras.Model(inputs =inputs,outputs= outputs)
    
    return model

In [6]:
mymodel = my_model()

print(mymodel.summary())

mymodel.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),
              optimizer = tf.keras.optimizers.Adam(lr=1e-4),
              metrics =["accuracy"]
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
tf.nn.relu (TFOpLambda)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256   

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [7]:
mymodel.fit(x_train,y_train,batch_size= 64,epochs = 3,verbose = 1)

Epoch 1/3
782/782 [==============================] - 449s 571ms/step - loss: 4.3056 - accuracy: 0.4348
Epoch 2/3
782/782 [==============================] - 434s 555ms/step - loss: 3.4240 - accuracy: 0.5674
Epoch 3/3
782/782 [==============================] - 454s 580ms/step - loss: 2.8914 - accuracy: 0.6308


In [8]:
mymodel.evaluate(x_test,y_test,batch_size=64,verbose=1)

157/157 [==============================] - 25s 158ms/step - loss: 2.6418 - accuracy: 0.6576


[2.6418004035949707, 0.6575999855995178]